In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle
import random
import requests
import io
import zipfile
from scipy.stats import norm
from typing import Optional


from simulation_helpers import*
from Simulation_models import*

In [5]:

def simulate_markov_chain(Q, T, k0=0, rng=None):
    if rng is None:
        rng = np.random.default_rng(0)
    Q = np.asarray(Q, float)
    K = Q.shape[0]
    if Q.shape != (K, K):
        raise ValueError("Q must be (K,K)")
    if not np.allclose(Q.sum(axis=1), 1.0):
        raise ValueError("Rows of Q must sum to 1.")
    k = np.empty(int(T), dtype=int)
    k[0] = int(k0)
    for t in range(1, int(T)):
        k[t] = rng.choice(K, p=Q[k[t - 1]])
    return k


def _as_3d_Phi(Phi, K, N):
    Phi = np.asarray(Phi, float)
    if Phi.ndim == 2:
        if Phi.shape != (N, N):
            raise ValueError("Phi (2D) must be (N,N)")
        return np.repeat(Phi[None, :, :], K, axis=0)
    if Phi.ndim == 3:
        if Phi.shape != (K, N, N):
            raise ValueError("Phi (3D) must be (K,N,N)")
        return Phi
    raise ValueError("Phi must be 2D (N,N) or 3D (K,N,N)")


def _expand_regime_param(x, K, fill_value):
    # ensures regime spec parameters have lengh k = regime
    if x is None:
        return np.full(K, fill_value, dtype=float)
    x = np.asarray(x, float).reshape(-1)
    if x.size < K:
        x = np.pad(x, (0, K - x.size), constant_values=float(fill_value))
    return x[:K]


def multivariate_t_eps_with_target_cov(rng, Sigma, df):
    """
    eps ~ multivariate Student-t(df) with scaling so that Cov(eps)=Sigma.
    We generate:
        z = L @ N(0,I)
        u ~ chi2(df)
        t_raw = z / sqrt(u/df)
        eps = t_raw * sqrt((df-2)/df)  (if df>2)
        https://en.wikipedia.org/wiki/Student%27s_t-distribution?utm_source=chatgpt.com
    """
    Sigma = np.asarray(Sigma, float)
    n = Sigma.shape[0]
    if Sigma.shape != (n, n):
        raise ValueError("Sigma must be square (N,N)")
    L = np.linalg.cholesky(Sigma + 1e-12 * np.eye(n))
    z = L @ rng.standard_normal(n)
    u = rng.chisquare(df)
    t_raw = z / np.sqrt(u / df)
    if df > 2.0:
        return np.sqrt((df - 2.0) / df) * t_raw
    return t_raw  # covariance is infinite if df<=2


def simulate_rs_var1_monthly_regimes_RS_SV_T(T_days,Q,c_list,Phi,Sigma_list,k0=0,burn_in_months=50,rng=None,
                                             start_date="2000-01-03",df_list=None,sv_rho=0.97,sv_sigma=0.20,
                                             logh_mu_list=None,return_h=False,
):

    if rng is None:
        rng = np.random.default_rng(0)

    Q = np.asarray(Q, float)
    c_list = np.asarray(c_list, float)
    Sigma_list = np.asarray(Sigma_list, float)

    K = Q.shape[0]
    if Q.shape != (K, K):
        raise ValueError("Q must be (K,K)")
    if not np.allclose(Q.sum(axis=1), 1.0):
        raise ValueError("Rows of Q must sum to 1.")

    if c_list.ndim != 2:
        raise ValueError("c_list must be (K,N)")
    if c_list.shape[0] != K:
        raise ValueError("c_list first dim must match K")
    N = c_list.shape[1]

    if Sigma_list.shape != (K, N, N):
        raise ValueError("Sigma_list must be (K,N,N) matching c_list")

    Phi_arr = _as_3d_Phi(Phi, K, N)

    df_list = _expand_regime_param(df_list, K, 8.0)
    logh_mu_list = _expand_regime_param(logh_mu_list, K, -1.5)

    # calendar
    T_days = int(T_days)
    sample_dates = pd.bdate_range(start_date, periods=T_days)
    start_month = sample_dates[0].to_period("M")
    end_month   = sample_dates[-1].to_period("M")

    sample_months = pd.period_range(start=start_month, end=end_month, freq="M")
    T_months = len(sample_months)

    burn_in_months = int(burn_in_months)
    full_months = pd.period_range(start=(start_month - burn_in_months), end=end_month, freq="M")
    TT_months = len(full_months)

    # monthly regimes
    k_month_full = simulate_markov_chain(Q, TT_months, k0=k0, rng=rng)

    # expand to business days (each month constant regime)
    all_dates_full = []
    k_days_full = []
    for m_idx, per in enumerate(full_months):
        month_start = per.to_timestamp(how="start")
        month_end   = per.to_timestamp(how="end")
        dts = pd.bdate_range(month_start, month_end)
        all_dates_full.append(dts)
        k_days_full.append(np.full(len(dts), int(k_month_full[m_idx]), dtype=int))

    all_dates_full = all_dates_full[0].append(all_dates_full[1:]) if len(all_dates_full) > 1 else all_dates_full[0]
    k_days_full = np.concatenate(k_days_full, axis=0)

    # simulate daily log returns
    T_full = len(all_dates_full)
    r_full = np.zeros((T_full, N), dtype=float)
    h_full = np.ones(T_full, dtype=float)

    # init log-vol at regime mean
    logh = float(logh_mu_list[int(k_days_full[0])])
    h_full[0] = float(np.exp(logh))

    sv_rho = float(sv_rho)
    sv_sigma = float(sv_sigma)

    for t in range(1, T_full):
        kt = int(k_days_full[t])
        df = float(df_list[kt])

        # SV(1) around regime mean
        mu_k = float(logh_mu_list[kt])
        logh = mu_k + sv_rho * (logh - mu_k) + sv_sigma * rng.standard_normal()
        h = float(np.exp(logh))
        h_full[t] = h

        eps = multivariate_t_eps_with_target_cov(rng, Sigma_list[kt], df=df)
        eps = np.sqrt(h) * eps  # Cov = h * Sigma_k

        r_full[t] = c_list[kt] + Phi_arr[kt] @ r_full[t - 1] + eps

    # drop burn-in by date indexing
    pos = all_dates_full.get_indexer(sample_dates)
    r_days = r_full[pos]
    k_days = k_days_full[pos]
    k_month = k_month_full[-T_months:] # so the original dates are aligned to the sample_months. no burn in here cause we already dropped by date indexing
    h_days = h_full[pos]

    if return_h:
        return r_days, k_days, k_month, sample_dates, sample_months, h_days
    return r_days, k_days, k_month, sample_dates, sample_months


# In-Sample Simulation Framework (No Saving)

This section simulates Markov regime-switching returns and evaluates strategies.
Outputs are displayed only (no files are saved).

In [10]:

class MarkovSimulationEnvironment:
    # TODO: loop over seed and test seeds, align with RL
    def __init__(self, Q, c_list, Phi, Sigma_list, regime_names,
                 T_days=252*30, seed=123, burn_in_months=50,
                 start_date="2000-01-03", k0=0, asset_names=None,
                 df_list=None, sv_rho=0.97, sv_sigma=0.20, logh_mu_list=None,
                 verbose: bool = False):
        """
        Q: Transition matrix (K x K)
        c_list: Drift per regime (K x N)
        Phi: AR(1) coefficient matrix (N x N) or (K x N x N), same
        Sigma_list: Covariance per regime (K x N x N)
        regime_names: List of regime names, align with Q and other parameters!
        df_list: Student-t df per regime (length K)
        sv_rho: persistence of log-vol (vol clustering)
        sv_sigma: innovation std of log-vol
        logh_mu_list: regime-specific mean level of log-vol (length K)
        """
        self.Q = np.asarray(Q, float)
        self.c_list = np.asarray(c_list, float)
        self.Phi = np.asarray(Phi, float)
        self.Sigma_list = np.asarray(Sigma_list, float)
        self.regime_names = regime_names
        self.T_days = T_days
        self.seed = seed
        self.burn_in_months = burn_in_months
        self.start_date = start_date
        self.k0 = k0
        self.verbose = verbose
        
        self.K = len(regime_names)
        self.N = self.c_list.shape[1]
        self.asset_names = asset_names or [f"Asset{i+1}" for i in range(self.N)]
        
        self.df_list = df_list
        self.sv_rho = sv_rho
        self.sv_sigma = sv_sigma
        self.logh_mu_list = logh_mu_list
        
        self.daily_returns_df = None
        self.monthly_returns_df = None
        self.daily_regimes = None
        self.monthly_regimes = None
        self.dates = None
        self.sample_months = None
        
    def _expand_regime_param(self, values, fill_value):
        values = np.asarray(values, float) if values is not None else None
        if values is None:
            return np.full(self.K, fill_value, dtype=float)
        if values.size < self.K:
            return np.pad(values, (0, self.K - values.size), constant_values=fill_value)
        return values[:self.K]
        
    def simulate(self):
        set_numpy_determinism(self.seed)
        rng = np.random.default_rng(self.seed)
    
        df_list = self._expand_regime_param(self.df_list, 8.0)
        logh_mu_list = self._expand_regime_param(self.logh_mu_list, -1.0)
        sv_rho = float(self.sv_rho)
        sv_sigma = float(self.sv_sigma)
    
        r_log, k_daily, k_month, dates, sample_months = simulate_rs_var1_monthly_regimes_RS_SV_T(
            T_days=self.T_days,
            Q=self.Q,
            c_list=self.c_list,
            Phi=self.Phi,
            Sigma_list=self.Sigma_list,
            k0=self.k0,
            burn_in_months=self.burn_in_months,
            rng=rng,
            start_date=self.start_date,
            df_list=df_list,
            sv_rho=sv_rho,
            sv_sigma=sv_sigma,
            logh_mu_list=logh_mu_list,
        )
    
        # Convert log returns to simple returns !!!!!
        r_simple = np.exp(r_log) - 1.0
    
        self.daily_returns_df = pd.DataFrame(
            r_simple,
            index=dates,
            columns=self.asset_names
        )
    
        self.daily_regimes = (
            pd.Series(k_daily, index=dates, name="regime_int")
            .map(lambda x: self.regime_names[int(x)])
        )
        self.daily_regimes.name = "regime"
    
        self.monthly_returns_df = compute_monthly_factor_returns_from_daily(
            self.daily_returns_df,
            factor_cols=self.asset_names
        )
    
        # Build monthly regimes indexed EXACTLY like monthly_returns_df.index!!!!!
        # k_month is aligned to sample_months (PeriodIndex, monthly)
        k_month_ser = pd.Series(k_month, index=sample_months, name="regime_int")
    
        idx = self.monthly_returns_df.index  # last trading day timestamps
        self.monthly_regimes = (
            k_month_ser
            .reindex(idx.to_period("M"))          # month period -> regime int
            .set_axis(idx)                        # index = last trading day timestamps
            .map(lambda x: self.regime_names[int(x)])  # int -> regime name !!!
        )
        self.monthly_regimes.name = "regime"
    
        self.dates = dates
        self.sample_months = sample_months
    
        if self.verbose:
            ok = self.monthly_regimes.index.equals(self.monthly_returns_df.index)
            print(f"  Simulated {len(self.daily_returns_df):,} days across {len(self.monthly_returns_df)} months")
            print(f"  Monthly regime index aligned to monthly returns: {ok}")
            print(f"  Daily regime distribution: {self.daily_regimes.value_counts().to_dict()}")
    
        return self
    

    
    def save_simulation_data(self, filepath):
        """Save all simulation data to pickle"""
        data = {
            'daily_returns': self.daily_returns_df,
            'monthly_returns': self.monthly_returns_df,
            'daily_regimes': self.daily_regimes,
            'monthly_regimes': self.monthly_regimes,
            'dates': self.dates,
            'sample_months': self.sample_months,
            'parameters': {
                'Q': self.Q,
                'c_list': self.c_list,
                'Phi': self.Phi,
                'Sigma_list': self.Sigma_list,
                'regime_names': self.regime_names,
                'seed': self.seed,
                'asset_names': self.asset_names,
                'df_list': self.df_list,
                'sv_rho': self.sv_rho,
                'sv_sigma': self.sv_sigma,
                'logh_mu_list': self.logh_mu_list
            },
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        if self.verbose:
            print(f" Saved simulation data to {filepath}")
        
    @staticmethod
    def load_simulation_data(filepath):
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        print(f" Loaded simulation data from {filepath}")
        return data

In [4]:
class StrategyBacktester:

    
    def __init__(self, env, verbose: bool = False):
        self.env = env
        self.results = {}
        self.verbose = verbose

    def _log(self, msg: str):
        if self.verbose:
            print(msg)
        
    def run_volatility_management(self, name="VolManagement", ridge=1e-8, gamma=5.0, c_tc=0.0021):
        self._log(f"\n[{name}] Running...")
        
        result = Markowitz_with_turnover_TC_diffobj(
            daily_factors=self.env.daily_returns_df,
            factor_cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            nonneg=True,
            c_tc=c_tc,
            use_drift_turnover=True,
            bounded_b=False  # Allow volatility timing via b parameters
        )
        
        # Compute daily returns from monthly weights
        # check this. do we scale with Mkt-rf?
        daily_port_returns = daily_portfolio_returns_from_monthly_weights(
            self.env.daily_returns_df,
            result['weights']
        )
        
        # Get weights by regime
        weights_by_regime = self._compute_weights_by_regime_markowitz(
            result['weights']
        )
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['portfolio_returns_net'],
            'weights_monthly': result['weights'],
            'turnover': result['turnover'],
            'costs': result['costs'],
            'a_params': result['a'],
            'b_params': result['b'],
            'weights_by_regime': weights_by_regime,
            'type': 'VolManagement'
        }
        
        self._log(f"   Vol-timing params (b): {result['b'].round(4).to_dict()}")
        return self
    def run_volatility_management_daily(self, name="VolManagement_daily",
                                   ridge=1e-8, gamma=3.0, c_tc=0.0001, vol_window=20):
        self._log(f"\n[{name}] Running...")

        result = Markowitz_with_turnover_TC_diffobj_daily(
            daily_factors=self.env.daily_returns_df,
            factor_cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            nonneg=True,
            c_tc=c_tc,
            use_drift_turnover=True,
            vol_window=vol_window,
            bounded_b=False
        )

        # These are DAILY objects:
        w_daily   = result["weights"]                # DataFrame, daily index
        rport_net = result["portfolio_returns_net"]  # Series, daily index

        # If you want weights-by-regime, use DAILY regimes (aligned on dates)

        self.results[name] = {
            "portfolio_returns_daily": rport_net,   
            "weights_daily": w_daily,
            "turnover_daily": result["turnover"],
            "costs_daily": result["costs"],
            "a_params": result["a"],
            "b_params": result["b"],
            "type": "VolManagement",
        }

        self._log(f"   Vol-timing params (b): {result['b'].round(4).to_dict()}")
        return self

    
    def run_markowitz(self, name="Markowitz", gamma=5.0, c_tc=0.0021, 
                        use_drift_turnover=True, bounded_b=True, ridge=1e-8):
        self._log(f"\n[{name}] Running...")
        
        result = Markowitz_with_turnover_TC_diffobj(
            daily_factors=self.env.daily_returns_df,
            factor_cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            nonneg=True,
            c_tc=c_tc,
            use_drift_turnover=use_drift_turnover,
            bounded_b=bounded_b # no volatility timing.
        )
        
        daily_port_returns = daily_portfolio_returns_from_monthly_weights(
            self.env.daily_returns_df,
            result['weights']
        )
        
        weights_by_regime = self._compute_weights_by_regime_markowitz(
            result['weights']
        )
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['portfolio_returns_net'],
            'weights_monthly': result['weights'],
            'turnover': result['turnover'],
            'costs': result['costs'],
            'a_params': result['a'],
            'b_params': result['b'],
            'weights_by_regime': weights_by_regime,
            'type': 'Markowitz'
        }
        
        self._log(f"  Avg monthly turnover: {result['turnover'].mean():.4f}")
        return self

    def run_equal_weight(self, name="EqualWeight"):
        self._log(f"\n[{name}] Running...")
        
        weights = pd.Series(
            1.0 / self.env.N, 
            index=self.env.asset_names
        )
        
        daily_port_returns = (self.env.daily_returns_df @ weights)
        monthly_port_returns = (1 + daily_port_returns).groupby(
            daily_port_returns.index.to_period("M")
        ).prod() - 1.0
        monthly_port_returns.index = monthly_port_returns.index.to_timestamp("M")
        
        weights_by_regime = pd.DataFrame(
            {regime: weights for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': monthly_port_returns,
            'weights': weights,
            'weights_by_regime': weights_by_regime,
            'type': 'EqualWeight'
        }
        
        return self

    def run_mv_simple(self, name="MV_Simple", gamma=5.0, ridge=1e-10, 
                        sum_to_one_constraint=True, long_only=True):
        self._log(f"\n[{name}] Running...")
        
        result = mv_simple(
            daily_ret=self.env.daily_returns_df,
            cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            sum_to_one_constraint=sum_to_one_constraint,
            long_only=long_only
        )
        
        weights_series = result['weights']
        daily_port_returns = (self.env.daily_returns_df @ weights_series)
        
        weights_by_regime = pd.DataFrame(
            {regime: weights_series for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['port_monthly'],
            'weights': weights_series,
            'weights_by_regime': weights_by_regime,
            'type': 'MV_Simple'
        }
        
        self._log(f"   Weights: {weights_series.round(4).to_dict()}")
        return self

    def run_mw_cvar_simple(self, name="MV_CVaR", beta=0.95, gamma=5.0,
                            sum_to_one_constraint=True, long_only=True, 
                            upper_bound=1.0, solver="ECOS"):
        self._log(f"\n[{name}] Running...")
        

        
        result = mw_cvar_simple(
            daily_ret=self.env.daily_returns_df,
            cols=self.env.asset_names,
            beta=beta,
            gamma=gamma,
            sum_to_one_constraint=sum_to_one_constraint,
            long_only=long_only,
            upper_bound=upper_bound,
            solver=solver,
            verbose=False
        )
        
        weights_series = result['weights']
        daily_port_returns = (self.env.daily_returns_df @ weights_series)
        
        weights_by_regime = pd.DataFrame(
            {regime: weights_series for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['port_monthly'],
            'weights': weights_series,
            'weights_by_regime': weights_by_regime,
            'cvar_loss': result['cvar_loss'],
            'type': 'MV_CVaR'
        }
        
        self._log(f"   Weights: {weights_series.round(4).to_dict()}")
        self._log(f"   CVaR (loss): {result['cvar_loss']:.6f}")
        return self

    def _compute_weights_by_regime_markowitz(self, weights_monthly):
        common = weights_monthly.index.intersection(self.env.monthly_regimes.index)
        
        regime_weights = []
        for regime in self.env.regime_names:
            mask = self.env.monthly_regimes.loc[common] == regime
            if mask.sum() == 0:
                continue
            
            avg_weights = weights_monthly.loc[common][mask].mean()
            
            weights = avg_weights.to_dict()
            weights['Regime'] = regime
            regime_weights.append(weights)
        
        return pd.DataFrame(regime_weights).set_index('Regime')

    def save_results(self, filepath):
        data = {
            'results': self.results,
            'environment': {
                'regime_names': self.env.regime_names,
                'asset_names': self.env.asset_names,
                'seed': self.env.seed
            }
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        if self.verbose:
            print(f"Saved backtest results to {filepath}")

## Run All Scenarios and Generate Comprehensive Tables



In [5]:


# MV_CVaR vs VolManagement 

regime_names = ["Bull", "Neutral", "Bear"] # ! align with the parameters as intended.
asset_names  = ["Asset1_Growth", "Asset2_Carry", "Asset3_Defensive"]

GAMMA   = 5.0 # difficult to interpret the gamma here in case of cvar.
seeds   = [53, 274, 1234, 89]
TIE_EPS = 1e-6




Phi_fixed = np.array([
    [0.15, 0.10, 0.10],
    [0.10, 0.15, 0.10],
    [0.10, 0.10, 0.15],
], dtype=float)
Phi_k = np.tile(Phi_fixed, (3, 1, 1))

c_base = np.array([
    [0.00120, 0.00020, 0.00070],  # Bull
    [0.00080, 0.00018, 0.00060],  # Neutral
    [0.00040, 0.00015, 0.00050],  # Bear
], dtype=float)

Sigma_base = np.array([
    # Bull
    [[0.00020000, 0.00008000, 0.00006300],
     [0.00008000, 0.00016000, 0.00005600],
     [0.00006300, 0.00005600, 0.00018000]],
    # Neutral
    [[0.00032400, 0.00013000, 0.00010800],
     [0.00013000, 0.00025600, 0.00009000],
     [0.00010800, 0.00009000, 0.00028900]],
    # Bear
    [[0.00062500, 0.00015000, -0.00010000],
     [0.00015000, 0.00040000,  0.00008000],
     [-0.00010000, 0.00008000, 0.00048400]],
], dtype=float)


# TAIL-SHOCK dominates: MV_CVaR should win 
# Interpretation:
#   - Crashes are big but short and no vol clustering
#   - MV_CVaR wins by structurally avoiding tail-exposed mixes -> lower realized vol

Q_tailshock = np.array([
    [0.997, 0.002, 0.001],  # Bull is very persistent
    [0.080, 0.900, 0.020],  # Neutral somewhat persistent
    [0.985, 0.010, 0.005],   # Bear is not persistent and switch immediately
], dtype=float)

df_tailshock = np.array([25.0, 10.0, 2.2], dtype=float)        # extreme Bear tails and decreasing gaussian per regime ( bull, neutral, bear)
logh_mu_tailshock = np.array([-3.0, -2.4, 0.35], dtype=float)   # Bear scale spikes
sv_rho_tailshock, sv_sigma_tailshock = 0.20, 0.90              # NOT persistent => timing weak, but the Q matrix is persistant and huge sigma


# VOL-TIMING dominates
# Interpretation:
#   - Volatility is persistent and regime-separated (clustering)
#   - Scaling exposure during high-vol periods reduces realized std a lot

Q_voltiming = np.array([
    [0.94, 0.05, 0.01],       # Persistent Bull
    [0.10, 0.80, 0.10],     # Neutral somewhat persistent   
    [0.02, 0.03, 0.95],      # Bear very persistent
], dtype=float)

# we need to specify these as well as model dependent. If we would have cash...
df_voltiming = np.array([25.0, 15.0, 12.0], dtype=float) # tail risk intensity.
logh_mu_voltiming = np.array([-4.0, -2.6, -0.8], dtype=float)  # big regime vol gaps
sv_rho_voltiming, sv_sigma_voltiming = 0.995, 0.25             # persistent => timing strong


targeted_scenarios = [
    dict(
        name="TailShockWorld_Sharpe",
        Q=Q_tailshock, c=c_base, Sigma=Sigma_base,
        df_list=df_tailshock, logh_mu_list=logh_mu_tailshock,
        sv_rho=sv_rho_tailshock, sv_sigma=sv_sigma_tailshock,
        winner_metric="Sharpe (ann.)",
        expected_winner="MV_CVaR",
        mv_cvar_beta=0.95,
    ),
    dict(
        name="VolTimingWorld_Sharpe",
        Q=Q_voltiming, c=c_base, Sigma=Sigma_base,
        df_list=df_voltiming, logh_mu_list=logh_mu_voltiming,
        sv_rho=sv_rho_voltiming, sv_sigma=sv_sigma_voltiming,
        winner_metric="Sharpe (ann.)",
        expected_winner="VolManagement",
        mv_cvar_beta=0.95,
    ),
]


targeted_results = {}
targeted_tables  = {}

for scen in targeted_scenarios:
    scenario_name   = scen["name"]
    Q_matrix        = scen["Q"]
    c_scenario      = scen["c"]
    Sigma_scenario  = scen["Sigma"]

    mv_cvar_beta    = scen["mv_cvar_beta"]
    winner_metric   = scen["winner_metric"]
    expected_winner = scen["expected_winner"]

    perf_tables = []

    for seed in seeds:
        env = MarkovSimulationEnvironment(
            Q=Q_matrix,
            c_list=c_scenario,
            Phi=Phi_k, # common 
            Sigma_list=Sigma_scenario,
            regime_names=regime_names,
            T_days=252 * 70,
            seed=seed,
            burn_in_months=50,
            start_date="2000-01-03",
            k0=0,
            asset_names=asset_names,
            verbose=False,

            df_list=scen["df_list"],
            sv_rho=scen["sv_rho"],
            sv_sigma=scen["sv_sigma"],
            logh_mu_list=scen["logh_mu_list"],
        )
        env.simulate() # initialize returns

        backtester = StrategyBacktester(env, verbose=False)

        # ONLY the two main strategies
        backtester.run_mw_cvar_simple("MV_CVaR", beta=mv_cvar_beta, gamma=GAMMA) # !!!!!!!!
        backtester.run_volatility_management("VolManagement", gamma=GAMMA, c_tc=0.0021) # !!!! nonneg= True, bounded_b = False and use_drift_turnover = True

        # "no timing" baseline
        backtester.run_markowitz(
            "VolManagement_Unconditional",
            gamma=GAMMA,
            c_tc=0.0021,
            use_drift_turnover=True,
            bounded_b=True
        )

        portfolios = {name: data["monthly_returns"] for name, data in backtester.results.items()}
        perf_table = make_table_for_portfolios(portfolios, periods_per_year=12)
        perf_tables.append(perf_table)

    common_index = perf_tables[0].index
    common_cols  = perf_tables[0].columns
    perf_tables  = [t.reindex(index=common_index, columns=common_cols) for t in perf_tables]
    mean_table = sum(perf_tables) / len(perf_tables)

    print("\n" + "-" * 60)
    print(f"MEAN MONTHLY PERFORMANCE TABLE (across {len(seeds)} seeds): {scenario_name}")
    print("-" * 60)
    print(mean_table.round(3).to_string())



    targeted_results[scenario_name] = {
        "mean_perf_table": mean_table,                  
        "winner_metric": winner_metric,
        "expected_winner": expected_winner,
        "seeds": seeds,
        "gamma": GAMMA,
        "mv_cvar_beta": mv_cvar_beta,
    }
    targeted_tables[scenario_name] = mean_table


Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']
Using factor columns: ['Asset1_Growth', 'Asset2_Carry', 'Asset3_Defensive']

------------------------------------------------------------
MEAN MONTHLY PERFORMANCE TABLE (across 4 seeds): TailShockWorld_Sharpe
------------------------------------------------------------
                          MV_CVaR  VolManagement  VolManagement_Unconditional
Ann. Mean (%)              38.144         42.451                       42.512
Ann. StdDev (%)             8.051         

In [6]:
# positive definite check

Sigma_k = np.array([
# Bull: moderate risk, mild diversification from A3
[[0.000220, 0.000080, -0.000030],
    [0.000080, 0.000100, -0.000010],
    [-0.000030,-0.000010, 0.000140]],

# Neutral: a bit more risk/correlation
[[0.000320, 0.000110, -0.000035],
    [0.000110, 0.000130, -0.000015],
    [-0.000035,-0.000015, 0.000170]],

# Bear: A1 risk explodes; A3 is a *true* hedge (strong negative cov with A1)
[[0.001500,-0.000150, -0.000650],
    [-0.000150,0.000220,  0.000020],
    [-0.000650,0.000020,  0.000500]],
], dtype=float)
def ensure_pd(S, eps=1e-10):
    """Return (is_pd, S_fixed). If not PD, adds diagonal jitter to make it PD."""
    S = 0.5 * (S + S.T)  # symmetrize
    try:
        np.linalg.cholesky(S)
        return True, S
    except np.linalg.LinAlgError:
        # shift diagonal so smallest eigenvalue becomes eps
        lam_min = np.min(np.linalg.eigvalsh(S))
        shift = (eps - lam_min) if lam_min < eps else eps
        S_fixed = S + shift * np.eye(S.shape[0])
        return False, S_fixed
        

ok, Sigma0 = ensure_pd(Sigma_k[0])
Sigma_k[0] = Sigma0
ok

True

In [ ]:
# daily Volatility
# 



# MV_CVaR vs VolManagement 

dfs_by_scenario = {"BB": nested_bull_bear, "BN": nested_bull_neutral, "NB": nested_neutral_bear}
REGIME_NAME = {0: "Bull", 1: "Neutral", 2: "Bear"}


regime_names = ["Bull", "Neutral", "Bear"] # ! align with the parameters as intended.
asset_names  = ["Asset1_Growth", "Asset2_Carry", "Asset3_Defensive"]

GAMMA   = 3.0 # decreased to 3 from 5
seeds   = [53, 274, 1234, 89] # training, do we need testing here? 
test_seeds = [1,2,3,4]
TIE_EPS = 1e-6


# daily log-return drifts
const_k = np.array([
    [ 0.00105,  0.00028, -0.00018],  # Bull: A1 strong carry; A3 costs (insurance premium)
    [ 0.00045,  0.00022, -0.00008],  # Neutral: mild carry; A3 still costs a bit
    [-0.00180,  0.00008,  0.00055],  # Bear: A1 crashy; A3 pays; A2 small positive
], dtype=float)

Phi_fixed = np.array([
    [0.12, 0.04, 0.02],   # A1 depends mostly on own lag
    [0.03, 0.10, 0.02],   # A2 smaller own persistence than A!
    [0.02, 0.03, 0.08],   # A3 lowest persistence (insurance-like), rare events
], dtype=float)

# Use the same Phi in all regimes to isolate the effect of:
# (i) drift differences c_k, (ii) covariance differences Sigma_k, (iii) SV/t differences (logh_mu, df, sv_rho/sigma)
Phi_k = np.tile(Phi_fixed, (3, 1, 1))

Sigma_k = np.array([
# Bull: moderate risk, mild diversification from A3
[[0.000220, 0.000080, -0.000030],
    [0.000080, 0.000100, -0.000010],
    [-0.000030,-0.000010, 0.000140]],

# Neutral: a bit more risk/correlation
[[0.000320, 0.000110, -0.000035],
    [0.000110, 0.000130, -0.000015],
    [-0.000035,-0.000015, 0.000170]],

# Bear: A1 risk explodes; A3 is a *true* hedge (strong negative cov with A1)
[[0.001500,-0.000150, -0.000650],
    [-0.000150,0.000220,  0.000020],
    [-0.000650,0.000020,  0.000500]],
], dtype=float)

df_list = np.array([18.0, 14.0, 8.0], dtype=float)         # tails exist everywhere, worse in Bear
logh_mu_list = np.array([-2.5, -2.2, -1.4], dtype=float)   # Bear higher vol level
sv_rho, sv_sigma = 0.995, 0.18                              # persistent--> forecastable


Q_bull_bear = np.array([
[0.92, 0.04, 0.04],   # Bull mostly stays Bull, sometimes Neutral/Bear
[0.15, 0.70, 0.15],   # Neutral can go either way
[0.05, 0.05, 0.90],   # Bear persistent, occasional exit
]   , dtype=float)

# (BN) Bull-Neutral scenario: Bear is rare, Bull/Neutral are persistent
Q_bull_neutral = np.array([
    [0.94, 0.05, 0.01],   # Bull -> mostly Bull, some Neutral, very rare Bear
    [0.08, 0.90, 0.02],   # Neutral persistent, small chance Bull/Bear
    [0.20, 0.20, 0.60],   # if Bear happens, it can exit (not too sticky here)
], dtype=float)

# (NB) Neutral-Bear scenario: more time in Neutral/Bear, Bull less dominant
Q_neutral_bear = np.array([
    [0.75, 0.20, 0.05],   # Bull less persistent, drifts into Neutral/Bear
    [0.05, 0.85, 0.10],   # Neutral persistent, sometimes Bear
    [0.03, 0.07, 0.90],   # Bear persistent
], dtype=float)

targeted_scenarios = [

    dict(
        name="VolTimingWorld_bull_bear",
        Q=Q_bull_bear, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
        winner_metric="Sharpe (ann.)",
        expected_winner="?",
        mv_cvar_beta=0.95,
    ),
       dict(
        name="VolTimingWorld_bull_neutral",
        Q=Q_bull_neutral, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
        winner_metric="Sharpe (ann.)",
        expected_winner="?",
        mv_cvar_beta=0.95,
    ),
       dict(
        name="VolTimingWorld_neutral_bear",
        Q=Q_neutral_bear, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
        winner_metric="Sharpe (ann.)",
        expected_winner="?",
        mv_cvar_beta=0.95,
    ),
]


targeted_results = {}
targeted_tables  = {}

for scen in targeted_scenarios:
    scenario_name   = scen["name"]
    Q_matrix        = scen["Q"]
    c_scenario      = scen["c"]
    Sigma_scenario  = scen["Sigma"]

    mv_cvar_beta    = scen["mv_cvar_beta"]
    winner_metric   = scen["winner_metric"]
    expected_winner = scen["expected_winner"]

    perf_tables = []

    for seed in seeds:
        # TODO save
        env = MarkovSimulationEnvironment(
            Q=Q_matrix,
            c_list=c_scenario,
            Phi=Phi_k, # common 
            Sigma_list=Sigma_scenario,
            regime_names=regime_names,
            T_days=252 * 30,
            seed=seed,
            burn_in_months=50,
            start_date="2000-01-03",
            k0=0,
            asset_names=asset_names,
            verbose=False,

            df_list=scen["df_list"],
            sv_rho=scen["sv_rho"],
            sv_sigma=scen["sv_sigma"],
            logh_mu_list=scen["logh_mu_list"],
        )
        env.simulate() # initialize returns

        backtester = StrategyBacktester(env, verbose=False)

        # ONLY the two main strategies
        backtester.run_volatility_management_daily("VolManagement", gamma=GAMMA, c_tc=0.0001,vol_window=42) # !!!! nonneg= True, bounded_b = False and use_drift_turnover = True



        portfolios = {name: data["portfolio_returns_daily"] for name, data in backtester.results.items()}
        perf_table = make_table_for_portfolios(portfolios, periods_per_year=252)
        perf_tables.append(perf_table)

    common_index = perf_tables[0].index
    common_cols  = perf_tables[0].columns
    perf_tables  = [t.reindex(index=common_index, columns=common_cols) for t in perf_tables]
    mean_table = sum(perf_tables) / len(perf_tables)

    print("\n" + "-" * 60)
    print(f"MEAN MONTHLY PERFORMANCE TABLE (across {len(seeds)} seeds): {scenario_name}")
    print("-" * 60)
    print(mean_table.round(3).to_string())



    targeted_results[scenario_name] = {
        "mean_perf_table": mean_table,                  
        "winner_metric": winner_metric,
        "expected_winner": expected_winner,
        "seeds": seeds,
        "gamma": GAMMA,
        "mv_cvar_beta": mv_cvar_beta,
    }
    targeted_tables[scenario_name] = mean_table



------------------------------------------------------------
MEAN MONTHLY PERFORMANCE TABLE (across 4 seeds): VolTimingWorld_bull_bear
------------------------------------------------------------
                          VolManagement
Ann. Mean (%)                     9.159
Ann. StdDev (%)                  11.644
Ann. SemiDev (%)                 10.716
CVaR 95% (%)                     -1.577
Avg DD (%)                       12.374
VaR 95% (%)                      -0.782
Sharpe (ann.)                     0.762
Sortino (ann.)                    0.856
Tail-Adj Sharpe (CVaR95)          5.793
Tail-Adj Sharpe (mVaR95)          9.164

------------------------------------------------------------
MEAN MONTHLY PERFORMANCE TABLE (across 4 seeds): VolTimingWorld_bull_neutral
------------------------------------------------------------
                          VolManagement
Ann. Mean (%)                    22.948
Ann. StdDev (%)                  16.882
Ann. SemiDev (%)                 17.696
CVa

In [6]:
# trainins seeds, test seeds 
def fit_volmanagement_ab_daily(
    daily_factors: pd.DataFrame,
    factor_cols,
    gamma=3.0,
    ridge=1e-8,
    c_tc=0.0001,
    vol_window=42,
    use_drift_turnover=True,
    bounded_b=False,
):
    """
    Train step: estimate a,b using Markowitz_with_turnover_TC_diffobj_daily on TRAIN data.
    Returns:a,b
    """
    res = Markowitz_with_turnover_TC_diffobj_daily(
        daily_factors=daily_factors,
        factor_cols=factor_cols,
        gamma=gamma,
        ridge=ridge,
        nonneg=True,
        c_tc=c_tc,
        use_drift_turnover=use_drift_turnover,
        vol_window=vol_window,
        bounded_b=bounded_b,
    )
    return res["a"], res["b"], res


def apply_volmanagement_ab_daily(
    daily_factors: pd.DataFrame,
    factor_cols,
    a: pd.Series,
    b: pd.Series,
    c_tc=0.0001,
    vol_window=42,
    use_avg_vol_proxy=True,
    market_vol_proxy=None,
    abs_eps=1e-10,
    use_drift_turnover=True,
):
    """
    Test step: apply fixed (a,b) to TEST data to get daily weights and daily net portfolio returns.

    """
    factor_cols = [c for c in factor_cols if (market_vol_proxy is None or c != market_vol_proxy)]

    X = daily_factors[factor_cols].copy().dropna(how="any")  # daily simple returns
    if X.empty:
        raise ValueError("No usable daily data after dropping NaNs.")

    # --- build vol signal s_t exactly like your daily optimizer does ---
    if (market_vol_proxy is not None) and (market_vol_proxy in daily_factors.columns):
        proxy = daily_factors.loc[X.index, market_vol_proxy].astype(float)
        s = proxy.rolling(int(vol_window)).std(ddof=1)
    else:
        if use_avg_vol_proxy:
            s = X.rolling(int(vol_window)).std(ddof=1).mean(axis=1)
        else:
            proxy = X.mean(axis=1)
            s = proxy.rolling(int(vol_window)).std(ddof=1)

    s = s.shift(1)  # one-day ahead
    valid = s.notna()

    Xv = X.loc[valid]
    sv = s.loc[valid].to_numpy().reshape(-1, 1)
    sv = np.maximum(sv, 1e-12)

    R = Xv.to_numpy()
    T, N = R.shape

    # align a,b to columns
    a_vec = a.reindex(Xv.columns).to_numpy().reshape(1, -1)
    b_vec = b.reindex(Xv.columns).to_numpy().reshape(1, -1)

    # theta_t = a + b / s_t
    theta = a_vec + (b_vec / sv)

    # fully invested normalization (same spirit as your eta_to_theta)
    denom = theta.sum(axis=1, keepdims=True)
    theta = theta / np.where(np.abs(denom) > 1e-12, denom, 1.0)

    weights = pd.DataFrame(theta, index=Xv.index, columns=Xv.columns)

    # turnover with drift (your definition)
    def smooth_abs(x):
        return np.sqrt(x * x + abs_eps)

    taus = np.zeros(T, dtype=float)
    if use_drift_turnover and T > 1:
        for t in range(1, T):
            w_prev_post = theta[t - 1]
            R_prev = R[t - 1]
            g = 1.0 + R_prev

            numer = w_prev_post * g
            denom = float(np.sum(numer))
            w_pre = numer / denom if abs(denom) > 1e-12 else w_prev_post

            w_target = theta[t]
            taus[t] = 0.5 * float(np.sum(smooth_abs(w_target - w_pre)))

    turnover = pd.Series(taus, index=Xv.index, name="turnover")
    costs = pd.Series(c_tc * taus, index=Xv.index, name="costs")

    port_ret_gross = (weights * Xv).sum(axis=1)
    port_ret_net = port_ret_gross - costs

    return {
        "weights": weights,
        "portfolio_returns_gross": port_ret_gross,
        "portfolio_returns_net": port_ret_net,
        "turnover": turnover,
        "costs": costs,
        "vol_signal": pd.Series(s.loc[valid].values, index=Xv.index, name="s_lag"),
    }


def run_volmanagement_oos_for_scenario(
    scenario_name: str,
    Q_matrix: np.ndarray,
    c_list: np.ndarray,
    Phi_k: np.ndarray,
    Sigma_list: np.ndarray,
    regime_names,
    asset_names,
    df_list,
    logh_mu_list,
    sv_rho,
    sv_sigma,
    train_seeds,
    test_seeds,
    train_T_days=252*50,
    test_T_days=252*20,
    burn_in_months_train=50,
    burn_in_months_test=50,
    start_date="2000-01-03",
    gamma=3.0,
    ridge=1e-8,
    c_tc=0.0001,
    vol_window=42,
):
    """
    Returns nested dict:
      out[train_seed][test_seed] = payload
    payload contains RL-like keys: port_ret_net, regime_days, weights, turnover, costs, a_params, b_params
    """
    out = {}

    for train_seed in train_seeds:
        env_tr = MarkovSimulationEnvironment(
            Q=Q_matrix, c_list=c_list, Phi=Phi_k, Sigma_list=Sigma_list,
            regime_names=regime_names, T_days=train_T_days, seed=train_seed,
            burn_in_months=burn_in_months_train, start_date=start_date, k0=0,
            asset_names=asset_names, verbose=False,
            df_list=df_list, sv_rho=sv_rho, sv_sigma=sv_sigma, logh_mu_list=logh_mu_list,
        ).simulate()

        a_hat, b_hat, _ = fit_volmanagement_ab_daily(
            daily_factors=env_tr.daily_returns_df,
            factor_cols=env_tr.asset_names,
            gamma=gamma,
            ridge=ridge,
            c_tc=c_tc,
            vol_window=vol_window,
            use_drift_turnover=True,
            bounded_b=False,
        )

        out[int(train_seed)] = {}

        for test_seed in test_seeds:
            # ---- TEST env ----
            env_te = MarkovSimulationEnvironment(
                Q=Q_matrix, c_list=c_list, Phi=Phi_k, Sigma_list=Sigma_list,
                regime_names=regime_names, T_days=test_T_days, seed=test_seed,
                burn_in_months=burn_in_months_test, start_date=start_date, k0=0,
                asset_names=asset_names, verbose=False,
                df_list=df_list, sv_rho=sv_rho, sv_sigma=sv_sigma, logh_mu_list=logh_mu_list,
            ).simulate()

            res_te = apply_volmanagement_ab_daily(
                daily_factors=env_te.daily_returns_df,
                factor_cols=env_te.asset_names,
                a=a_hat, b=b_hat,
                c_tc=c_tc,
                vol_window=vol_window,
                use_avg_vol_proxy=True,
                market_vol_proxy=None,
                use_drift_turnover=True,
            )

            r = res_te["portfolio_returns_net"]
            # align regimes to r index, map to ints for RL-like regime_days
            reg_names = env_te.daily_regimes.reindex(r.index)
            reg_to_int = {name: i for i, name in enumerate(env_te.regime_names)}
            k = reg_names.map(lambda x: reg_to_int.get(x, np.nan)).dropna().astype(int)

            # align lengths safely
            common_idx = r.index.intersection(k.index)
            r = r.loc[common_idx]
            k = k.loc[common_idx]

            payload = {
                "port_ret_net": r.to_numpy(),
                "regime_days": k.to_numpy(),
                "weights": res_te["weights"],
                "turnover": res_te["turnover"],
                "costs": res_te["costs"],
                "a_params": a_hat,
                "b_params": b_hat,
            }

            out[int(train_seed)][int(test_seed)] = payload

    return out



def run_all_scenarios_volmanagement_oos(
    targeted_scenarios,
    Phi_k,
    regime_names,
    asset_names,
    train_seeds,
    test_seeds,
    train_T_days=252*50,
    test_T_days=252*20,
    burn_in_months_train=50,
    burn_in_months_test=50,
    start_date="2000-01-03",
    gamma=3.0,
    ridge=1e-8,
    c_tc=0.0001,
    vol_window=42,
):
    """
    Returns dict keyed by scenario short code (e.g., "BB","BN","NB") or name.
    Structure is compatible with a modified summarizer (no tau).
    """
    out = {}
    for scen in targeted_scenarios:
        name = scen["name"]
        out[name] = run_volmanagement_oos_for_scenario(
            scenario_name=name,
            Q_matrix=scen["Q"],
            c_list=scen["c"],
            Phi_k=Phi_k,
            Sigma_list=scen["Sigma"],
            regime_names=regime_names,
            asset_names=asset_names,
            df_list=scen["df_list"],
            logh_mu_list=scen["logh_mu_list"],
            sv_rho=scen["sv_rho"],
            sv_sigma=scen["sv_sigma"],
            train_seeds=train_seeds,
            test_seeds=test_seeds,
            train_T_days=train_T_days,
            test_T_days=test_T_days,
            burn_in_months_train=burn_in_months_train,
            burn_in_months_test=burn_in_months_test,
            start_date=start_date,
            gamma=gamma,
            ridge=ridge,
            c_tc=c_tc,
            vol_window=vol_window,
        )
    return out



def summarize_path_metrics_volmgmt_oos(
    oos_by_scenario,
    periods_per_year=252,
    rf_annual=0.0,
    target=0.0,
    alpha=0.95,
    use_key="port_ret_net",
    REGIME_NAME=None,
    policy_name="VolManagement",
):
    """
    oos_by_scenario[scenario_name][train_seed][test_seed] = payload
    payload has: port_ret_net (np), regime_days (np)
    """
    rows = []

    for scenario, d_train in oos_by_scenario.items():
        for train_seed, d_test in d_train.items():
            for test_seed, payload in d_test.items():
                r = np.asarray(payload.get(use_key, []), float)
                k = np.asarray(payload.get("regime_days", []), int)

                n = min(r.size, k.size)
                if n < 2:
                    continue
                r = r[:n]
                k = k[:n]

                base = {
                    "Scenario": scenario,
                    "Policy": policy_name,
                    "TrainSeed": int(train_seed),
                    "TestSeed": int(test_seed),
                }

                # overall
                m_all = portfolio_stats_paper_style(
                    r, periods_per_year=periods_per_year, rf_annual=rf_annual, target=target, alpha=alpha
                )
                rows.append({**base, "Regime": "All", "N_obs": int(n), **m_all})

                # by regime
                if REGIME_NAME is not None:
                    for reg_id, reg_name in REGIME_NAME.items():
                        mask = (k == reg_id)
                        r_reg = r[mask]
                        if r_reg.size < 2:
                            continue
                        m = portfolio_stats_paper_style(
                            r_reg, periods_per_year=periods_per_year, rf_annual=rf_annual, target=target, alpha=alpha
                        )
                        rows.append({**base, "Regime": reg_name, "N_obs": int(r_reg.size), **m})

    return pd.DataFrame(rows)



In [21]:
# --- TRAIN / TEST seed split (RL-style) ---
train_seeds = [53, 274, 1234, 89]   # fit (a,b) here
test_seeds  = [1, 2, 3, 4]          # evaluate out-of-sample here

# --- common experiment settings ---
GAMMA      = 3.0
RIDGE      = 1e-8
C_TC       = 1e-4
VOL_WINDOW = 42

TRAIN_T_DAYS = 252 * 30
TEST_T_DAYS  = 252 * 20

BURN_IN_TRAIN = 50
BURN_IN_TEST  = 50

START_DATE = "2000-01-03"

REGIME_NAME = {0: "Bull", 1: "Neutral", 2: "Bear"}


regime_names = ["Bull", "Neutral", "Bear"] # ! align with the parameters as intended.
asset_names  = ["Asset1_Growth", "Asset2_Carry", "Asset3_Defensive"]

# daily log-return drifts
const_k = np.array([
    [ 0.00105,  0.00028, -0.00018],  # Bull: A1 strong carry; A3 costs (insurance premium)
    [ 0.00045,  0.00022, -0.00008],  # Neutral: mild carry; A3 still costs a bit
    [-0.00180,  0.00008,  0.00055],  # Bear: A1 crashy; A3 pays; A2 small positive
], dtype=float)

Phi_fixed = np.array([
    [0.12, 0.04, 0.02],   # A1 depends mostly on own lag
    [0.03, 0.10, 0.02],   # A2 smaller own persistence than A!
    [0.02, 0.03, 0.08],   # A3 lowest persistence (insurance-like), rare events
], dtype=float)

# Use the same Phi in all regimes to isolate the effect of:
# (i) drift differences c_k, (ii) covariance differences Sigma_k, (iii) SV/t differences (logh_mu, df, sv_rho/sigma)
Phi_k = np.tile(Phi_fixed, (3, 1, 1))

Sigma_k = np.array([
# Bull: moderate risk, mild diversification from A3
[[0.000220, 0.000080, -0.000030],
    [0.000080, 0.000100, -0.000010],
    [-0.000030,-0.000010, 0.000140]],

# Neutral: a bit more risk/correlation
[[0.000320, 0.000110, -0.000035],
    [0.000110, 0.000130, -0.000015],
    [-0.000035,-0.000015, 0.000170]],

# Bear: A1 risk explodes; A3 is a *true* hedge (strong negative cov with A1)
[[0.001500,-0.000150, -0.000650],
    [-0.000150,0.000220,  0.000020],
    [-0.000650,0.000020,  0.000500]],
], dtype=float)

df_list = np.array([18.0, 14.0, 8.0], dtype=float)         # tails exist everywhere, worse in Bear
logh_mu_list = np.array([-2.5, -2.2, -1.4], dtype=float)   # Bear higher vol level
sv_rho, sv_sigma = 0.995, 0.18                              # persistent--> forecastable


Q_bull_bear = np.array([
[0.92, 0.04, 0.04],   # Bull mostly stays Bull, sometimes Neutral/Bear
[0.15, 0.70, 0.15],   # Neutral can go either way
[0.05, 0.05, 0.90],   # Bear persistent, occasional exit
]   , dtype=float)

# (BN) Bull-Neutral scenario: Bear is rare, Bull/Neutral are persistent
Q_bull_neutral = np.array([
    [0.94, 0.05, 0.01],   # Bull -> mostly Bull, some Neutral, very rare Bear
    [0.08, 0.90, 0.02],   # Neutral persistent, small chance Bull/Bear
    [0.20, 0.20, 0.60],   # if Bear happens, it can exit (not too sticky here)
], dtype=float)

# (NB) Neutral-Bear scenario: more time in Neutral/Bear, Bull less dominant
Q_neutral_bear = np.array([
    [0.75, 0.20, 0.05],   # Bull less persistent, drifts into Neutral/Bear
    [0.05, 0.85, 0.10],   # Neutral persistent, sometimes Bear
    [0.03, 0.07, 0.90],   # Bear persistent
], dtype=float)

targeted_scenarios = [
    dict(
        short="BB",
        name="VolTimingWorld_bull_bear",
        Q=Q_bull_bear, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
    ),
    dict(
        short="BN",
        name="VolTimingWorld_bull_neutral",
        Q=Q_bull_neutral, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
    ),
    dict(
        short="NB",
        name="VolTimingWorld_neutral_bear",
        Q=Q_neutral_bear, c=const_k, Sigma=Sigma_k,
        df_list=df_list, logh_mu_list=logh_mu_list,
        sv_rho=sv_rho, sv_sigma=sv_sigma,
    ),
]
# --- INVOKE: run OOS (train a,b on train_seeds, test on test_seeds) for all scenarios ---

oos_by_scenario = {}

for scen in targeted_scenarios:
    scen_key = scen["short"]   # "BB","BN","NB"
    print(f"\n=== Running OOS VolManagement for {scen_key}: {scen['name']} ===")

    oos_by_scenario[scen_key] = run_volmanagement_oos_for_scenario(
        scenario_name=scen["name"],
        Q_matrix=scen["Q"],
        c_list=scen["c"],
        Phi_k=Phi_k,
        Sigma_list=scen["Sigma"],
        regime_names=regime_names,
        asset_names=asset_names,
        df_list=scen["df_list"],
        logh_mu_list=scen["logh_mu_list"],
        sv_rho=scen["sv_rho"],
        sv_sigma=scen["sv_sigma"],
        train_seeds=train_seeds,
        test_seeds=test_seeds,
        train_T_days=TRAIN_T_DAYS,
        test_T_days=TEST_T_DAYS,
        burn_in_months_train=BURN_IN_TRAIN,
        burn_in_months_test=BURN_IN_TEST,
        start_date=START_DATE,
        gamma=GAMMA,
        ridge=RIDGE,
        c_tc=C_TC,
        vol_window=VOL_WINDOW,
    )




=== Running OOS VolManagement for BB: VolTimingWorld_bull_bear ===

=== Running OOS VolManagement for BN: VolTimingWorld_bull_neutral ===

=== Running OOS VolManagement for NB: VolTimingWorld_neutral_bear ===


In [17]:
oos_by_scenario['BB'][53].keys()

dict_keys([1, 2, 3, 4])

In [19]:
oos_by_scenario['BB'][53][1].keys()

dict_keys(['port_ret_net', 'regime_days', 'weights', 'turnover', 'costs', 'a_params', 'b_params'])

In [22]:
path_df = summarize_path_metrics_volmgmt_oos(
    oos_by_scenario=oos_by_scenario,
    periods_per_year=252,
    rf_annual=0.0,
    target=0.0,
    alpha=0.95,
    use_key="port_ret_net",
    REGIME_NAME=REGIME_NAME,
    policy_name="VolManagement",
)

# regime-only table (Bull/Neutral/Bear) with N_obs weights, RL-style
metric_cols = [
    "Ann. Mean (%)","Ann. StdDev (%)","Ann. SemiDev (%)",
    "CVaR 95% (%)","Avg DD (%)","VaR 95% (%)",
    "Sharpe (ann.)","Sortino (ann.)",
    "Tail-Adj Sharpe (CVaR95)","Tail-Adj Sharpe (mVaR95)",
]

reg_only = path_df[path_df["Regime"].isin(["Bull","Neutral","Bear"])].copy()

regime_weighted = weighted_group_mean(
    reg_only,
    group_cols=["Scenario","Policy","Regime"],
    weight_col="N_obs",
    metric_cols=metric_cols
)

regime_weighted

,Scenario,Policy,Regime,N_paths,SumWeights,Ann. Mean (%),Ann. StdDev (%),Ann. SemiDev (%),CVaR 95% (%),Avg DD (%),VaR 95% (%),Sharpe (ann.),Sortino (ann.),Tail-Adj Sharpe (CVaR95),Tail-Adj Sharpe (mVaR95)
0,BB,VolManagement,Bear,16,29888.0,-5.343866,12.784185,11.271612,-1.801153,32.000235,-0.993173,-0.971004,-0.993015,-5.977009,-9.500442
1,BB,VolManagement,Bull,16,40620.0,13.595699,9.200950,9.359131,-1.286846,2.890865,-0.682242,1.662540,1.615773,11.612296,47.333449
2,BB,VolManagement,Neutral,16,9460.0,22.953094,17.674949,12.016484,-1.616615,2.269996,-0.679950,1.312088,1.766424,13.119176,32.625381
3,BN,VolManagement,Bear,16,1920.0,-32.631286,24.960830,25.575528,-3.595130,15.944289,-2.705947,-2.347793,-1.883230,-13.014638,-16.604239
4,BN,VolManagement,Bull,16,49552.0,27.668264,10.993802,11.547891,-1.572001,2.728747,-0.907405,2.609551,2.474718,18.303841,30.511563
5,BN,VolManagement,Neutral,16,28496.0,6.463987,14.340634,14.951475,-2.226880,17.430801,-1.279542,0.479435,0.490493,3.217632,6.211465
6,NB,VolManagement,Bear,16,37788.0,2.023876,14.839174,13.540171,-2.094781,23.421746,-1.220806,-0.089067,-0.090147,-0.386750,-0.489254
7,NB,VolManagement,Bull,16,8864.0,3.576947,12.252562,12.173608,-1.825653,9.336218,-0.917732,0.207454,0.240954,1.711899,5.614368
8,NB,VolManagement,Neutral,16,33316.0,14.021006,12.692834,10.120905,-1.439270,4.120735,-0.777974,1.029675,1.250042,8.797216,9.746060
